In [1]:
import sys
!conda install --prefix {sys.prefix} -y -c pytorch pytorch

Solving environment: ...working... done

# All requested packages already installed.





==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.3.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.3.0




In [2]:
import sys
!{sys.executable} -m pip install transformers

In [3]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

C:\Users\Weber\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [4]:
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelForCausalLM.from_pretrained('distilgpt2')

In [5]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [6]:
generator("Two roads diverged in a yellow wood, and")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Two roads diverged in a yellow wood, and police told them two other motorists jumped into a ditch in an apparent attempt to cut traffic.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'}]

In [7]:
generator("Two roads diverged in a yellow wood, and")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Two roads diverged in a yellow wood, and the man who was behind the scene of that accident has been identified as a 37-year-old from Gersfield. Authorities believe a 17-year-old boy was able to run away with'

And then you can generate text with the pipeline. The first argument is the prompt; any remaining parameters will be forwarded to the model's `.generate()` method.

In [65]:
generator("Two roads diverged in a yellow",
          max_length=100)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Two roads diverged in a yellow Ford GT and the Audi A3 GT, where a pair of people were reported missing in the busy Chico, South Carolina town of Carrizo County.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'}]

As a reminder, to get the actual generated text, use indexing to get the value of the dictionary in the list returned from the pipeline:

In [66]:
generator("Two roads diverged in a yellow",
          max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Two roads diverged in a yellow haze of clouds over the north of the country, causing the death of 14 people and one woman on the road. "Nobody was hurt," a man said. "At least six people were killed. On Wednesday morning, no one was injured, so a couple were injured in the blast," the mayor said. "It seemed that some people were killed. We had to find the way home. We were very much in shock. It was very hard to do as'

In [8]:
my_poem=generator("Two roads diverged in a yellow",
          max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [69]:
my_poem.replace("e", "EEEEEE")

'Two roads divEEEEEErgEEEEEEd in a yEEEEEEllow BMW S-SEEEEEEriEEEEEEs.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMt-T-D-N-R-T-T-E-N-R\n\n\n\n\n\n\n\n\n\n\n\n\n'

## Controlling the model

By default, the `distilgpt2` model samples from the possible next tokens, weighted by the probability assigned to that token. This strategy leads to text that shows a good deal of variety, but there are strategies that we can use and parameters that we can tweak to exert a little more control over the model's output. In this section, I show a few of these strategies.

### The magic of the prompt

Transformer models are often able to follow up on cues you give about the desired content and style of the text in the prompt itself. The smaller transformer models aren't especially good at this, but it's still worth playing around with. For example, to get `distilgpt2` to generate something that looks like a movie review:

In [70]:
print(generator("My review of The Road Not Taken, the Movie:", max_length=100)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


My review of The Road Not Taken, the Movie: a New Movie from Hollywood›..
The Road Not Taken, the Movie: a New Movie from Hollywood›
The Road Not Taken, the Movie: a New Movie from Hollywood›
The Road Not Taken, the Movie: a New Movie from Hollywood›
The Road Not Taken, the Movie: a New Movie from Hollywood›
If you haven't read the movie so far already, here's the top


You can also generate dialogues and interview transcripts:

In [72]:
print(generator("Allison: I took the road less traveled by.\nRobert Frost:", max_length=100)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Allison: I took the road less traveled by.
Robert Frost: I was going about the same thing. My whole life was about getting out the doors. That was what I wanted me to do. I wanted my car to be as safe and as safe as mine. And that's when I did exactly the opposite.
Joss Whedon: I had a lot of fun. I was on a plane and I looked at him and said to me that I love his work. So I


Poetry facts:

In [73]:
print(generator("My favorite facts about poetry:\n\n1.",
                max_length=100)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


My favorite facts about poetry:

1. The last time you read the song, you're going to remember that there was a time when no one wanted you to read the poem.
2. The number of times you're going to read the verse, you're going to remember that, 'B.S.W.'"
3. The number of times you're going to read the verse, you're going to remember that, 'W.'N.B.," said the poet.


In general, this kind of prompting works best with texts that are likely to have a lot of representation in the training corpus.

### Sampling with temperature

As mentioned above, the `distilgpt2` model, by default, picks the next token at random, weighted by the probability that the model assigns to the word. To demonstrate how this works, let's imagine that the model only has five tokens in its vocabulary (instead of 50,000+). A schematic illustration of those probabilities might look like this:

    prompt: Whose woods these are I think I
    probabilities:
        know -> 0.5
        knew -> 0.2
        smell -> 0.15
        see -> 0.1
        am -> 0.05


The probabilities will add up to `1.0`. A probability of `0.5` indicates that the token has a 50% probability of coming next; a probability of 0.2 means that the token has a 20% probability of coming next, etc. Here are those probabilities represented in Python as two lists—one with the words, and one with the probabilities that correspond to those words by index:

In [75]:
tokens = ['know', 'knew', 'smell', 'see', 'am']
probs = [0.5, 0.2, 0.15, 0.1, 0.05]

By default, to select the next token, the generation code picks from this list weighted by probability. The code to do this with PyTorch looks like this:

In [76]:
index = torch.multinomial(torch.tensor(probs), 1).item()
print(tokens[index])

know


You don't have to worry about the specifics of this code—I'm just using it to demonstrate how the sampling process works. Run the code a few times and you'll see that about half the time you get "knew"—the token with the highest probability. Running the code in a loop makes this a bit easier to see:

In [77]:
for i in range(10):
    index = torch.multinomial(torch.tensor(probs), 1).item()
    print(tokens[index])

know
smell
knew
smell
see
know
smell
knew
know
know


The generation process has a parameter called *temperature*, which lets you shift the probability distribution of the next token before it's sampled. If the temperature parameter is `1.0`, then sampling will proceed as normal, with the tokens weighted by their estimated probability. If the temperature parameter is less than `1.0`, then tokens that were already probable will get *more* probable. If the temperature parameter is greater than `1.0`, then the probabilities start to even out, approaching a uniform distribution (meaning that no token is more likely to be chosen than any other). To demonstrate this, I've written some code below that applies temperature to the probabilities defined above, and shows the resulting changes:

In [78]:
for temperature in [0.1, 0.35, 1.0, 2.0, 50.0]:
    modified = torch.softmax(
        torch.log(torch.tensor(probs)) / temperature, dim=-1)
    print(f"temperature {temperature:0.02f}")
    for tok, prob in zip(tokens, modified):
        print(tok.ljust(6), "→", f"{prob:0.002f}")
    print()

temperature 0.10
know   → 1.00
knew   → 0.00
smell  → 0.00
see    → 0.00
am     → 0.00

temperature 0.35
know   → 0.90
knew   → 0.07
smell  → 0.03
see    → 0.01
am     → 0.00

temperature 1.00
know   → 0.50
knew   → 0.20
smell  → 0.15
see    → 0.10
am     → 0.05

temperature 2.00
know   → 0.34
knew   → 0.21
smell  → 0.19
see    → 0.15
am     → 0.11

temperature 50.00
know   → 0.20
knew   → 0.20
smell  → 0.20
see    → 0.20
am     → 0.20



You can see that at temperature `1.0`, the probabilities are identical to the original. At temperature `0.35`, the probability of the most likely token has been boosted, but the other tokens still have a small chance of occurring. At temperature `0.1`, only the most likely token has a chance of being selected. At temperature `2.0`, the most likely token is still the most likely, but the probabilities of the other tokens have been boosted in comparison; at temperature `50.0`, no token is considered to be more likely than any other.

To apply temperature sampling to the model when generating text, pass the `temperature` parameter to the pipeline, like so:

In [79]:
generator("Two roads diverged in a yellow",
          temperature=0.1,
          max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Two roads diverged in a yellow light, and the police were called to the scene.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

Low temperatures generally produce predictable, repetitive results. Here's an attempt with high temperature:

In [80]:
generator("Two roads diverged in a yellow",
          temperature=4.0,
          max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Two roads diverged in a yellow BMW i1.'

The higher temperature example produces less likely sequences of words, so the text is a bit livelier—sometimes at the cost of coherence.

Adjusting the temperature can be useful when you want the text to be more or less "weird." It can be helpful to adjust the temperature downward when you feel as though the model is producing text that is a bit too unpredictable; it can be helpful to adjust upward when you want to model to take more unexpected turns when generating.

### Top-k sampling

By default, the generation process only selects from the top 50 most probable tokens at each step. This is called "top-k filtering." Because of top-k filtering, you're not likely to sample truly unusual tokens even when the temperature is high. You can adjust the threshold for top-k filtering with the `top_k` parameter of the model. For example, adjusting `top_k` to the number of items in the vocabulary ensures that every token gets its chance:

In [82]:
generator("Two roads diverged in a yellow",
          top_k=tokenizer.vocab_size,
          max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Two roads diverged in a yellow plume, coming from a maintenance spot on his chain, The Express.\n\n\n\n\n\n\n\n\n\n\n\nTri Lowe Police say he brought a cellphone in for a safety check to check who was making a threat to the property after he woke up on the street on Wednesday afternoon due to a fire.\n\nThe incident focused on the offence by hitting a Mazda 2016sized car.\n\nUPI'

Using this with a temperature greater than `1.0` can yield some unusual turns of phrase:

In [83]:
generator("Two roads diverged in a yellow",
          top_k=tokenizer.vocab_size,
          temperature=1.2,
          max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Two roads diverged in a yellow Mazda V-7 however driver Kim Whenerman vs. G differences might make the difference since Red assumes CPS can eat a heavier proportion of fuel compared with road racing cars – although Driving Do Cute uses class C sales confer less loads, this used 2 litres toatisfactor that poll to shopping district total, probably less nor much latter traffic density sow confusion and and vacancies on Tuesday As is often the case with CPS significant interventions, the GM-CCS met with friends'

On the other extreme, setting the `top_k` value to `1` ensures that *only* the most likely token is chosen at each step. This is the same thing as ["greedy decoding"](https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.greedy_search):

In [84]:
generator("Two roads diverged in a yellow",
          top_k=1,
          max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Two roads diverged in a yellow light, and the police were called to the scene.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

Playing around with `top_k` and `temperature` in tandem is a good way to make adjustments to the texture of your generated text.

### Logit warping: Exclude "bad" words

The `.generate()` method has a parameter called `bad_words_ids`, which causes the model to zero out the probabilities of tokens associated with words that you pass in. The intended use of this feature is to stop the model from generating offensive or harmful words. But we can also repurpose it for poetic purposes. For example, in the cell below, I make the model complete the prompt "It was a dark and stormy" *without* using the words "night" or "day":

In [50]:
generator("It was a dark and stormy",
          bad_words_ids=tokenizer([" night", " day"]).input_ids)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"It was a dark and stormy storm that set off the town of Nizke. The village of Nizke was deserted and abandoned. The village's people had fled, but they did not understand it. Then when they could finally reach N"

The syntax for specifying the "bad words" is to call the tokenizer on a list of words that you want to exclude, and then get the `.input_ids` attribute of the value returned from calling the tokenizer. This yields a list of lists that looks like this:

In [51]:
tokenizer(["Allison", "Parrish"]).input_ids

[[3237, 1653], [47, 3258, 680]]

Note that I used ` night` and ` day` as the words, with leading spaces—this is necessary because I ended the prompt without whitespace, so the model is likely to generate a token with leading whitespace at the next step. I've found that the `bad_words_ids` parameter works best if your list of words includes versions both with and without whitespace.

Here's another example: getting the model to complete a prompt without using any forms of the verb *to be*:

In [52]:
generator("Once upon a time,",
          bad_words_ids=tokenizer(
              ["be", " be",
               "am", " am",
               "are", " are",
               "is", " is",
               "was", " was",
               "were", " were"]).input_ids,
          max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Once upon a time, on the other end of the planet at the top of a galaxy, it became apparent that the star, the Milky Way, had the same gravitational background as the stars in this study. When those planets moved at great distances from the outside, they made much larger changes to their properties. However, once they reached the core of the Milky Way they began to move farther and farther in the opposite direction, and that distance gained increased further. Finally, before reaching the core of the'

You can also create a list of token IDs that you want to exclude on the fly. In the following example, I make a list of token IDs that have the letter `e` in them, and pass that list to the `bad_words_ids` parameter:

In [53]:
forbidden_ids = []
for key, val in tokenizer.get_vocab().items():
    if 'e' in key:
        forbidden_ids.append([val]) # needs to be a list of lists
print(generator("Last month, I",
          bad_words_ids=forbidden_ids,
          max_length=100)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Last month, I saw a lot of this, and I think it shows just how hard it is to avoid using Bitcoin just by looking at how far off that block diagram can go for Bitcoin's full functionality. First off, I had to go through a lot of data mining, which is why I want to start to look at a small part of why Bitcoin is a cash-only, anonymous, and anonymous Bitcoin.

For my first half hour and hour of work, I had to wait


### Fine-tuning a model

"Fine-tuning" is a way of slightly modifying a model by training it a few extra steps on a corpus of your choice. This process adjusts the probabilities of the model so that it more closely reflects the probabilities of the source text you train it on. Fine-tuning models with Transformers is a little bit tricky! First, you'll need to install Hugging Face's `datasets` package:

In [ ]:
import sys
!{sys.executable} -m pip install datasets

And then import it:

In [54]:
import datasets

You'll want to select a text file to fine-tune the model on. Fine-tuning works best on large amounts of text, but fine-tuning is also very slow if you're not using a GPU. For demonstration purposes, I create a special version of [Frankenstein](https://www.gutenberg.org/ebooks/84) that contains only the first 20000 characters, and save it to a file:

In [55]:
with open("84-0-20k.txt", "w") as fh:
    fh.write(open("84-0.txt").read()[:20000])

Then I load this text file as my fine-tuning dataset:

In [56]:
training_data = datasets.load_dataset('text', data_files="84-0-20k.txt")

Using custom data configuration default-8396e67af902a1a6


Dataset text downloaded and prepared to /Users/allison/.cache/huggingface/datasets/text/default-8396e67af902a1a6/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


Now, there's a bunch of obligatory processing that we need to do to the data in order to prepare it for the model. This is boilerplate stuff, which I'm not going to go into in detail. If you want details, consult Hugging Face's [fine-tuning language models notebook](https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb).

First, we tokenize the text:

In [57]:
tokenizer.pad_token = tokenizer.eos_token
tokenized_training_data = training_data.map(
    lambda x: tokenizer(x['text']),
    remove_columns=["text"]
)

Then we break the tokenized text up into batches of tokens:

In [58]:
block_size = 64
# magic from https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
lm_training_data = tokenized_training_data.map(
    group_texts,
    batched=True,
    batch_size=200
)

Now we import the `Trainer` class, which implements a training loop.

In [59]:
from transformers import Trainer, TrainingArguments

Running the following cell creates the `Trainer` object. The `output_dir` parameter specifies a directory where your fine-tuned model will be saved. The `num_train_epochs` sets how many "epochs" the trainer will run; one epoch is one iteration over the entire dataset. More epochs is better, but even one epoch can significantly change the way the model generates text.

In [60]:
trainer = Trainer(model=model,
                  train_dataset=lm_training_data['train'],
                  args=TrainingArguments(
                      output_dir='distilgpt2-finetune-frankenstein20k',
                      num_train_epochs=1,
                      do_train=True,
                      do_eval=False
                  ),
                  tokenizer=tokenizer)

Finally, the cell below will start the training process. If you're running this on a computer without a GPU, it will take a while. You can open this notebook on [Google Colab](http://colab.research.google.com/) if you want and take advantage of the free GPU that Google lets you use.

In [61]:
trainer.train()

***** Running training *****
  Num examples = 67
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=9, training_loss=4.9099070231119795, metrics={'train_runtime': 56.9456, 'train_samples_per_second': 1.177, 'train_steps_per_second': 0.158, 'total_flos': 2107446755328.0, 'train_loss': 4.9099070231119795, 'epoch': 1.0})

Running the cell below will save the model to disk:

In [62]:
trainer.save_model()

Saving model checkpoint to distilgpt2-finetune-frankenstein20k
Configuration saved in distilgpt2-finetune-frankenstein20k/config.json
Model weights saved in distilgpt2-finetune-frankenstein20k/pytorch_model.bin
tokenizer config file saved in distilgpt2-finetune-frankenstein20k/tokenizer_config.json
Special tokens file saved in distilgpt2-finetune-frankenstein20k/special_tokens_map.json


Now you can generate with the fine-tuned model! The fine-tuning process modifies the model in-place, so the `pipeline` you created before will make use of the fine-tuned model. (Note that if you want to get the original `distilgpt2` back, you'll need to reload it with the `.from_pretrained()` method, as demonstrated at the top of the notebook.)

In [64]:
generator("Two roads diverged in a yellow", max_length=100)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Two roads diverged in a yellow.\n\n\n\nAt a glance, I could not say what I was. I tried to hold my breaths, and felt a sense of peace that seemed to me an odd thing. I tried to keep my composure.\nThe distance between them was far beyond my comprehension, and so, then, seemed to the most of my life. As if I had always wished to stay home, as though the weather was different, I had no space for my own'

You can see that fine-tuning on even a small dataset produces big changes in the model.

If you want to use your fine-tuned model in another project, use the same syntax that we used above to load `distilgpt2`—just replace `distilgpt2` with the name of the directory where you saved your model:

In [65]:
my_tokenizer = AutoTokenizer.from_pretrained('distilgpt2-finetune-frankenstein20k')
my_model = AutoModelForCausalLM.from_pretrained('distilgpt2-finetune-frankenstein20k')

Didn't find file distilgpt2-finetune-frankenstein20k/added_tokens.json. We won't load it.
loading file distilgpt2-finetune-frankenstein20k/vocab.json
loading file distilgpt2-finetune-frankenstein20k/merges.txt
loading file distilgpt2-finetune-frankenstein20k/tokenizer.json
loading file None
loading file distilgpt2-finetune-frankenstein20k/special_tokens_map.json
loading file distilgpt2-finetune-frankenstein20k/tokenizer_config.json
loading configuration file distilgpt2-finetune-frankenstein20k/config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "do_sample": true,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "max_length": 50,
  "model_type": "gp

Now generate with it:

In [66]:
my_generator = pipeline("text-generation", model=my_model, tokenizer=my_tokenizer)

In [68]:
my_generator("Two roads diverged in a yellow")[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Two roads diverged in a yellowish haze, and there were only two clear roadways crossing east of the river. As the sky fell and the ground was darkened, the wind blew a terrible wind; but it soon returned to the east where he'